In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.rcParams['axes.linewidth'] = .5
plt.rcParams['xtick.major.width'] = .5
plt.rcParams['ytick.major.width'] = .5
plt.rcParams['xtick.minor.width'] = .0
plt.rcParams['ytick.minor.width'] = .0
plt.rcParams['axes.labelpad'] = 1
plt.rcParams['axes.titlepad'] = 3
fontsize=6

In [3]:
import torch
import os, sys
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
qualitative = os.path.abspath('')
e_vae_proj = os.path.dirname(qualitative)
disvae = os.path.dirname(e_vae_proj)
if not disvae in sys.path: sys.path.append(disvae)
from utils.datasets import get_dataloaders
from disvae.utils.modelIO import load_model

# constraint
cons='rec'

In [4]:
seed = 1234
nlat = 10
batchs = 64
epochs = 200
lrs = [0.0005, 0.001]
increments = [4, 2]
if cons == "rec":
    epsilon = 30
else:
    epsilon = 0.08

In [5]:
def traverse_latent_space(model_path, data):
    # load model & get (reconstructions, latent posteriors, latent_sample) for data
    model = load_model('path', gpu_id=-1)
    with torch.no_grad():
        recons, lat_dist, lat_sample = model(data)
    
    # vary the latent sample based on the given latent posteriors & get reconstructions
    imgs = []
    for i, z0 in enumerate(lat_sample):
        for z in np.linspace(lat_dist[i][0] - np.exp(lat_dist[i][1]), lat_dist[i][0] - np.exp(lat_dist[i][1]), 10):
            lat_sample[i] = z
            imgs.append(self.decoder(lat_sample[i]))
            lat_sample[i] = z0
    return imgs

In [6]:
def plot_traversals(model_path, data):
    # get traversal imgs
    imgs = traverse_latent_space(model_path, data)
    
    # plot grid of varying latent_sample (x), latent_variable (y)
    figure, axes = plt.figure(dpi=200)

In [ ]:
# get visualisation data
n_data = 1
seed = 14
torch.manual_seed(seed)
dataloader = get_dataloaders('celeba', batch_size=n_data*n_data)
data = next(iter(dataloader))[0]
img_data = np.zeros((64 * n_data, 64 * n_data))
for i in range(n_data):
    for j in range(n_data):
        img_data[(i * 64):(i * 64 + 64), (j * 64):(j * 64 + 64)] = data[i * n_data + j, 0, :, :].numpy()

In [7]:
celeba_evae_models = [
    f"evae_celeba/{cons}_{epochs}ep_z{nlat}_e{epsilon}_s{seed}_lr{lr}_incr{increment}"
    for lr in lrs
    for increment in increments
]

celeba_evae_imgs_example = traverse_latent_space(celeba_evae_models[0], )